In [1]:
!pip install llama-index-readers-s3 llama-index-vector-stores-chroma -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.4 MB/s eta 0:00:

In [2]:
!pip install --upgrade llama-index -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 11.3 MB/s eta 0:00:00


In [3]:
!pip show llama-index-readers-s3

Name: llama-index-readers-s3
Version: 0.4.0
Summary: llama-index readers s3 integration
Home-page: 
Author: Your Name
Author-email: you@example.com
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: llama-index-core, llama-index-readers-file, s3fs
Required-by: 


In [3]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.s3 import S3Reader
from google.colab import userdata

In [4]:
open_ai_key = userdata.get('open_ai_api')
aws_access_key = userdata.get('aws_access_key_id')
aws_secret_key = userdata.get('aws_secret_key')

import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [5]:
# Initialize S3Reader without the need for a custom reader class
loader = S3Reader(
    bucket="diary-log",
    key="user-data/user_id=dennis.json",
    aws_access_id=aws_access_key,
    aws_access_secret=aws_secret_key,
    #file_extractor={".json": json_extractor},  # Use custom extractor
)

documents = loader.load_data()

Since S3 files are in JSON format, need to handle them before indexing

In [6]:
# Handle JSON entries from S3
def process_json_entries(documents):
    """
    Process the raw JSON entries from s3 into a format suitable for indexing
    """

    import json
    from llama_index.core.schema import Document
    # Extract the text content from the MediaResource
    json_text = documents[0].text_resource.text

    # Split into individual JSON entries and parse
    entries = [json.loads(entry) for entry in json_text.strip().split('\n') if entry]

    # Create formatted documents for each entry
    processed_docs = []
    for entry in entries:
        formatted_text = f"""
        Date: {entry['logDate']}
        Theme: {entry['themeEntry']}
        Study Time: {entry['studyTime']} hours
        Entry: {entry['diaryEntry']}
        """
        doc = Document(
            text=formatted_text,
            metadata={
                'date': entry['logDate'],
                'theme': entry['themeEntry'],
                'study_time': entry['studyTime']
            }
        )
        processed_docs.append(doc)
    return processed_docs

In [7]:
docs = process_json_entries(documents)
docs[0]

Document(id_='3701db44-3a8e-49cb-835c-7a5b84e9307d', embedding=None, metadata={'date': '2025-01-01', 'theme': 'Snowboarding', 'study_time': '2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='\n        Date: 2025-01-01\n        Theme: Snowboarding\n        Study Time: 2 hours\n        Entry: Today was my first time snowboarding. I spent most of the session getting comfortable with the board and learning to balance on flat terrain. I practiced getting up after falling, which happened a lot! I focused on learning how to glide smoothly and controlling my speed. It was a lot harder than I thought, but I\\u2019m excited to come back tomorrow. I also learned how to properly position my feet on the board to avoid any strain. By the end of the session, I was feeling more confident, even though I kept falling.\n        ', path=None, url=N

In [8]:
# Create Index
index = VectorStoreIndex.from_documents(docs)

In [11]:
from llama_index.core import Settings


Settings.chunk_size = 100
Settings.chunk_overlap = 10

index = VectorStoreIndex.from_documents(docs)

query_engine = index.as_query_engine()


response = query_engine.query("What sport did i do?")

print(response)


You went snowboarding.


# VectorDB

In [12]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.create_collection("s3_entries")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [13]:
index = VectorStoreIndex.from_documents(
    docs, storage_context=storage_context
)
query_engine = index.as_query_engine()
chat_engine = index.as_chat_engine(
                        chat_mode="openai",
                        verbose=True
                        )
response = chat_engine.query("sumamrize my snowboarding experience and recommend what to do next")


response

Added user message to memory: sumamrize my snowboarding experience and recommend what to do next
=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "summarize my snowboarding experience"}
Got output: You focused on learning how to glide smoothly, control your speed, position your feet properly on the board, and balance on flat terrain. Despite falling multiple times, you felt more confident by the end of the session and are excited to continue learning.

=== Calling Function ===
Calling function: query_engine_tool with args: {"input": "recommend what to do next in snowboarding"}
Got output: Continue practicing carving to make smoother turns and reduce skidding. Work on mastering stopping techniques further. Consider practicing on steeper sections to challenge yourself. Additionally, focus on improving your confidence and balance by continuing to work on controlling your speed and gliding smoothly.



Response(response='Your snowboarding experience involved focusing on learning how to glide smoothly, control your speed, position your feet properly on the board, and balance on flat terrain. Despite falling multiple times, you felt more confident by the end of the session and are excited to continue learning.\n\nTo enhance your snowboarding skills further, it is recommended to continue practicing carving to make smoother turns and reduce skidding. Work on mastering stopping techniques and consider practicing on steeper sections to challenge yourself. Additionally, focus on improving your confidence and balance by continuing to work on controlling your speed and gliding smoothly.', source_nodes=[NodeWithScore(node=TextNode(id_='3fb1a773-bbf4-42db-ae88-a1466955c02c', embedding=None, metadata={'date': '2025-01-01', 'theme': 'Snowboarding', 'study_time': '2'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(nod

In [15]:
print(response)

Your snowboarding experience involved focusing on learning how to glide smoothly, control your speed, position your feet properly on the board, and balance on flat terrain. Despite falling multiple times, you felt more confident by the end of the session and are excited to continue learning.

To enhance your snowboarding skills further, it is recommended to continue practicing carving to make smoother turns and reduce skidding. Work on mastering stopping techniques and consider practicing on steeper sections to challenge yourself. Additionally, focus on improving your confidence and balance by continuing to work on controlling your speed and gliding smoothly.


In [14]:
# Inspecting the retrieved document
print("Retrieved Document Text:")
print(response.source_nodes[0].text)  # Text from the retrieved document

print("\nDocument Metadata:")
for key, value in response.source_nodes[0].metadata.items():
    print(f"{key}: {value}")

print("\nDocument Score:")
print(response.source_nodes[0].score)


Retrieved Document Text:
I focused on learning how to glide smoothly and controlling my speed. It was a lot harder than I thought, but I\u2019m excited to come back tomorrow. I also learned how to properly position my feet on the board to avoid any strain. By the end of the session, I was feeling more confident, even though I kept falling.

Document Metadata:
date: 2025-01-01
theme: Snowboarding
study_time: 2

Document Score:
0.7563043835109347


In [ ]:
def